In [ ]:
# Librerías
# !pip install imaplib email pandas openpyxl

In [2]:
import imaplib
import email
from email.header import decode_header
import os
from datetime import datetime, timedelta
import re
import pandas as pd
import pickle
import html
import locale
from email.utils import parsedate_to_datetime
from pytz import timezone
import logging

# Configuración de la cuenta
IMAP_SERVER = "imap-mail.outlook.com"
EMAIL_ACCOUNT = "facturas_gpf@outlook.com"
APP_PASSWORD = "lleibtocysmvsnko"

# Configurar el log
logging.basicConfig(filename='procesamiento_emails.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Conectar al servidor IMAP
try:
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL_ACCOUNT, APP_PASSWORD)
    logging.info('Conexión al servidor IMAP exitosa')
except Exception as e:
    logging.error(f'Error al conectar al servidor IMAP: {e}')
    raise

mail.select("inbox")

# Obtener las fechas de hoy, ayer y mañana en formato inglés
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
date_today = datetime.now().strftime("%d-%b-%Y")
date_yesterday = (datetime.now() - timedelta(1)).strftime("%d-%b-%Y")
date_tomorrow = (datetime.now() + timedelta(1)).strftime("%d-%b-%Y")
locale.setlocale(locale.LC_TIME, '')  # Restaurar la configuración regional por defecto

# Buscar correos de ayer, hoy y mañana
try:
    status_today, messages_today = mail.search(None, f'(ON {date_today})')
    status_yesterday, messages_yesterday = mail.search(None, f'(ON {date_yesterday})')
    status_tomorrow, messages_tomorrow = mail.search(None, f'(ON {date_tomorrow})')
    logging.info(f'Búsqueda de correos exitosa, estado hoy: {status_today}, estado ayer: {status_yesterday}, estado mañana (por cambio zona horaria): {status_tomorrow}')
except Exception as e:
    logging.error(f'Error al buscar correos: {e}')
    raise

messages = messages_today[0].split() + messages_yesterday[0].split() + messages_tomorrow[0].split()
logging.info(f'Total de correos encontrados: {len(messages)}')

processed_emails = []

# Leer archivo pickle para evitar duplicados
try:
    with open('processed_emails.pkl', 'rb') as f:
        processed_data = pickle.load(f)
    logging.info('Archivo pickle de correos procesados cargado correctamente')
except FileNotFoundError:
    processed_data = {}
    logging.info('Archivo pickle no encontrado, iniciando con datos vacíos')

def extract_data_from_xml(xml_path):
    try:
        with open(xml_path, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        with open(xml_path, 'r', encoding='ISO-8859-1', errors='replace') as file:
            content = file.read()

    content = html.unescape(content)
    
    patterns = {
        'ruc': r'<ruc>(.*?)<\/ruc>',
        'estab': r'<estab>(.*?)<\/estab>',
        'ptoEmi': r'<ptoEmi>(.*?)<\/ptoEmi>',
        'secuencial': r'<secuencial>(.*?)<\/secuencial>',
        'total_sin_impuestos': r'<totalSinImpuestos>(.*?)<\/totalSinImpuestos>',
        'fecha_emision': r'<fechaEmision>(.*?)<\/fechaEmision>',
        'nombre_comercial': r'<razonSocial>(.*?)<\/razonSocial>',
        'compania': r'<razonSocialComprador>(.*?)<\/razonSocialComprador>'
    }
    
    extracted_data = {key: (match.group(1) if (match := re.search(pattern, content)) else '') for key, pattern in patterns.items()}
    
    subtotales = {}
    iva_values = {}

    # Buscar en la etiqueta "detalle" -> "impuestos" -> "impuesto"
    detalle_pattern = r'<detalle>(.*?)<\/detalle>'
    detalle_matches = re.findall(detalle_pattern, content, re.DOTALL)

    for detalle in detalle_matches:
        impuestos_pattern = r'<impuesto>(.*?)<\/impuesto>'
        impuestos_matches = re.findall(impuestos_pattern, detalle, re.DOTALL)

        for impuesto in impuestos_matches:
            base_imponible_pattern = r'<baseImponible>(.*?)<\/baseImponible>'
            tarifa_pattern = r'<tarifa>(.*?)<\/tarifa>'
            valor_pattern = r'<valor>(.*?)<\/valor>'

            base_imponible = re.search(base_imponible_pattern, impuesto)
            tarifa = re.search(tarifa_pattern, impuesto)
            valor = re.search(valor_pattern, impuesto)

            if base_imponible and tarifa and valor:
                base_imponible = float(base_imponible.group(1))
                tarifa = float(tarifa.group(1))
                valor = float(valor.group(1))

                if tarifa not in subtotales:
                    subtotales[tarifa] = 0
                if tarifa not in iva_values:
                    iva_values[tarifa] = 0

                subtotales[tarifa] += base_imponible
                iva_values[tarifa] += valor

    if not subtotales and not iva_values:
        total_impuestos_pattern = r'<totalImpuesto>(.*?)<\/totalImpuesto>'
        total_impuestos_matches = re.findall(total_impuestos_pattern, content, re.DOTALL)

        for total_impuesto in total_impuestos_matches:
            base_imponible_pattern = r'<baseImponible>(.*?)<\/baseImponible>'
            tarifa_pattern = r'<tarifa>(.*?)<\/tarifa>'
            valor_pattern = r'<valor>(.*?)<\/valor>'

            base_imponible = re.search(base_imponible_pattern, total_impuesto)
            tarifa = re.search(tarifa_pattern, total_impuesto)
            valor = re.search(valor_pattern, total_impuesto)

            if base_imponible and tarifa and valor:
                base_imponible = float(base_imponible.group(1))
                tarifa = float(tarifa.group(1))
                valor = float(valor.group(1))

                if tarifa not in subtotales:
                    subtotales[tarifa] = 0
                if tarifa not in iva_values:
                    iva_values[tarifa] = 0

                subtotales[tarifa] += base_imponible
                iva_values[tarifa] += valor

    return extracted_data, subtotales, iva_values

def decode_payload(part):
    payload = part.get_payload(decode=True)
    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    for enc in encodings:
        try:
            return payload.decode(enc)
        except UnicodeDecodeError:
            continue
    return payload.decode('utf-8', errors='replace')

def format_date(date_str):
    date_obj = parsedate_to_datetime(date_str)
    ecuador_tz = timezone('America/Guayaquil')
    localized_date = date_obj.astimezone(ecuador_tz)
    return localized_date.strftime('%d/%m/%Y'), localized_date.strftime('%H:%M:%S')

def extract_oc_numbers(body):
    oc_pattern = r'OC[\s.]*[:]*[\s]*(\d+(?:[\s,-]+[\s]*\d+)*)'
    oc_matches = re.findall(oc_pattern, body, re.IGNORECASE)
    logging.debug(f'OCs encontradas: {oc_matches}')

    oc_numbers = []
    for match in oc_matches:
        numbers = re.split(r'[\s,-]+', match.strip())
        oc_numbers.extend(numbers)

    result = ' - '.join(filter(None, oc_numbers))
    logging.debug(f'Números de OC extraídos: {result}')
    return result

def extract_original_sender(body):
    # Patrón para buscar el remitente original en el cuerpo del correo reenviado
    original_sender_pattern = r'De:\s*([^<]+)<([^>]+)>'
    match = re.search(original_sender_pattern, body, re.IGNORECASE)
    if match:
        name = match.group(1).strip()
        email = match.group(2).strip()
        return f"{name} <{email}>"
    return None

def process_message(msg_num):
    status, msg_data = mail.fetch(msg_num, "(RFC822)")
    if status != 'OK':
        logging.error(f'Error al obtener el mensaje número: {msg_num}')
        return
    
    msg = email.message_from_bytes(msg_data[0][1])
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding if encoding else "utf-8")
    sender = msg.get("From")
    date = msg.get("Date")
    formatted_date, formatted_time = format_date(date)
    logging.info(f'Procesando mensaje de: {sender}, fecha: {formatted_date}, hora: {formatted_time}, asunto: {subject}')
    
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body += decode_payload(part)
    else:
        body = decode_payload(msg)
    
    # Intentar extraer el remitente original del cuerpo del correo
    original_sender = extract_original_sender(body)
    if original_sender:
        sender = original_sender

    oc_numbers = extract_oc_numbers(body)
    if oc_numbers:
        secuencial = re.search(r'FACTURA[\s-]*(\d+-\d+-\d+)', subject, re.IGNORECASE)
        if secuencial:
            secuencial = secuencial.group(1)
            if (secuencial, oc_numbers) not in processed_data:
                processed_data[(secuencial, oc_numbers)] = True
                email_data = {
                    "remitente": sender,
                    "fecha": formatted_date,
                    "hora": formatted_time,
                    "asunto": subject,
                    "ocs": oc_numbers,
                    "Factura": secuencial
                }
                for part in msg.walk():
                    if part.get_content_maintype() == "multipart":
                        continue
                    if part.get("Content-Disposition") is None:
                        continue
                    filename = part.get_filename()
                    if filename and filename.endswith(".xml"):
                        xml_filename = filename
                        xml_part = part
                    elif filename and filename.endswith(".pdf"):
                        pdf_filename = filename
                        pdf_part = part
                if xml_filename and pdf_filename:
                    xml_base = os.path.splitext(xml_filename)[0]
                    pdf_base = os.path.splitext(pdf_filename)[0]
                    if xml_base == pdf_base:
                        folder_name = subject.replace("/", "_")
                        if not os.path.isdir(folder_name):
                            os.mkdir(folder_name)
                        xml_path = os.path.join(folder_name, xml_filename)
                        pdf_path = os.path.join(folder_name, pdf_filename)
                        with open(xml_path, "wb") as f:
                            f.write(xml_part.get_payload(decode=True))
                        with open(pdf_path, "wb") as f:
                            f.write(pdf_part.get_payload(decode=True))
                        xml_info, subtotales, iva_values = extract_data_from_xml(xml_path)
                        subtotal_columns = {f"subtotal_{tarifa}%": subtotal for tarifa, subtotal in subtotales.items()}
                        iva_columns = {f"IVA_{tarifa}%": iva for tarifa, iva in iva_values.items()}
                        email_data.update({
                            "ruc": xml_info['ruc'],
                            "total_sin_impuestos": float(xml_info['total_sin_impuestos']),
                            "fecha_emision": xml_info['fecha_emision'],
                            "nombre_comercial": xml_info['nombre_comercial'],
                            "compania": xml_info['compania'],
                        })
                        email_data.update(subtotal_columns)
                        email_data.update(iva_columns)
                        processed_emails.append(email_data)
                        logging.info(f'Email data procesada y agregada: {email_data}')
                        if os.path.exists(xml_path):
                            os.remove(xml_path)
                        if os.path.exists(pdf_path):
                            os.remove(pdf_path)
                        if not os.listdir(folder_name):
                            os.rmdir(folder_name)
                    else:
                        logging.info(f'Mensaje sin archivos adjuntos XML y PDF correspondientes: {subject}')
                else:
                    logging.info(f'No se encontraron OCs en el msj: {subject}')


# Procesar los mensajes
for msg_num in messages:
    process_message(msg_num)

# Crear DataFrame si hay correos procesados
if processed_emails:
    df = pd.DataFrame(processed_emails)
    column_order = ["remitente", "fecha", "hora", "asunto", "ocs", "Factura", "ruc", "total_sin_impuestos"]
    subtotal_iva_columns = [col for col in df.columns if col.startswith("subtotal_") or col.startswith("IVA_")]
    column_order.extend(subtotal_iva_columns)
    column_order.extend([col for col in df.columns if col not in column_order])
    
    # Verificar que las columnas existen en el DataFrame antes de reordenar
    column_order = [col for col in column_order if col in df.columns]
    
    df = df[column_order]
    df.to_excel("processed_emails.xlsx", index=False)
else:
    logging.info('No se encontraron correos para procesar en las fechas especificadas.')
    # Crear un archivo Excel vacío con las columnas necesarias
    df = pd.DataFrame(columns=["remitente", "fecha", "hora", "asunto", "ocs", "Factura", "ruc", "total_sin_impuestos"])
    df.to_excel("processed_emails.xlsx", index=False)

# Guardar el archivo pickle actualizado
with open('processed_emails.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

# Cerrar la conexión IMAP y el log
mail.close()
mail.logout()
logging.shutdown()

: 

In [ ]:
def actualizar_tabla_excel_y_limpieza(ruta_excel_salida, access_token):
    # Verifica si el archivo existe, si no, crea un archivo vacío con una hoja inicial
    inicializar = not os.path.exists(ruta_excel_salida)
    if inicializar:
        with pd.ExcelWriter(ruta_excel_salida, engine='openpyxl') as writer:
            pd.DataFrame().to_excel(writer, sheet_name='Hoja_Temporal', index=False)  # Crea una hoja temporal vacía

    # Asumiendo que `current_dir`, `pickle_file`, y `csv_oc_pendientes` están definidos en el ámbito global o importados previamente
    archivos = glob.glob(os.path.join(current_dir, 'OCS', '**', '*.xml'), recursive=True)
    dataframe_total = pd.DataFrame()

    with open(pickle_file, 'rb') as f:
        facturas_procesadas = pickle.load(f)
    df_oc_pendientes = pd.read_csv(csv_oc_pendientes)

    for ruta_archivo in archivos:
        informacion, subtotales, iva_values = extraer_informacion_de_archivo(ruta_archivo)
        factura = f"{informacion['estab']}-{informacion['ptoEmi']}-{informacion['secuencial']}"
        oc = informacion['OC']

        if oc not in facturas_procesadas:
            facturas_procesadas[oc] = True
            descripcion = informacion['descripciones']

            # Preparar las nuevas columnas
            subtotal_0 = subtotales.get(0, 0)
            tarifas_no_0 = ";".join([str(tarifa) for tarifa in subtotales if tarifa != 0])
            subtotales_impuesto_no_0 = ";".join([str(subtotales[tarifa]) for tarifa in subtotales if tarifa != 0])
            iva_no_0 = ";".join([str(iva_values[tarifa]) for tarifa in iva_values if tarifa != 0])

            dataframe_temporal = pd.DataFrame({
                'Autorizacion': [informacion["autorizacion"]],
                'RUC': [informacion['ruc']],
                'Tercero': [informacion['Tercero']],
                'Nombre Comercial': [informacion['nombre_comercial']],
                'Compañía': [informacion['compania']],
                'Centro de Costo': [informacion['Centro de Costo']],
                'Nombre Farmacia': [informacion['Nombre Farmacia']],
                'Factura': [factura],
                'Fecha': [informacion['fecha_formateada']],
                'OC': [oc],
                'Frecuencia facturación': [informacion['Frecuencia facturación']],
                'Descripcion': [descripcion],
                'Subtotal 0%': [subtotal_0],
                'Tarifa': [tarifas_no_0],
                'Subtotales Impuesto': [subtotales_impuesto_no_0],
                'IVA': [iva_no_0]
            })


In [ ]:
import pickle

with open('processed_emails.pkl', 'rb') as f:
    processed_data = pickle.load(f)

print(processed_data)